## dbt Cloud API Example for listing runs for a given job (with _pagination_)
- The pagination allows for returning more than 100 results in proper order


___

### Step One: Important dependencies 

In [1]:
import requests
from datetime import datetime
import json

### Step Two: Define account and job information

In [ ]:
# dbt Cloud Account Information
dbt_cloud_api_token = "dbtc_12343547457456745"
dbt_cloud_account_id = 12345

# Set up the headers for authentication
api_headers = {
    'Authorization': f'Token {dbt_cloud_api_token}',
    'Content-Type': 'application/json'
}

# dbt Cloud Job Information
dbt_cloud_job_id = 106102

### Step Three: Build function to call dbt Cloud API and get runs for a job using pagination

In [38]:
def list_dbt_cloud_runs_for_a_job(api_token, account_id, job_id, max_runs=100):
    """
    List runs from the dbt Cloud API with pagination.
    
    Parameters:
    - api_token (str): Your dbt Cloud API token.
    - account_id (int): The account ID for your dbt Cloud project.
    - job_id (int): The job ID to list runs for.
    - max_runs (int, optional): Maximum number of runs to retrieve (default: 100).
    
    Returns:
    - list: A list of runs for the specified job ID.
    """
    
    url = f"https://cloud.getdbt.com/api/v2/accounts/{account_id}/runs/"
    headers = {
        "Authorization": f"Token {api_token}"
    }
    runs = []
    offset = 0
    limit = 100  # dbt Cloud's maximum limit per request
    
    while len(runs) < max_runs:
        params = {
            "job_definition_id": job_id,
            "limit": limit,
            "offset": offset,
            "order_by": "-id"
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Failed to retrieve runs: {response.status_code}, {response.text}")
            break
        
        data = response.json()
        batch_runs = data.get("data", [])
        runs.extend(batch_runs)
        
        # Check if we have reached the last page
        pagination = data.get("extra", {}).get("pagination", {})
        if len(batch_runs) < limit or len(runs) >= max_runs or pagination.get("count", 0) < limit:
            break
        
        # Update offset to fetch the next page
        offset += limit
    
    # Trim the runs list to the specified max_runs
    return runs[:max_runs]

In [ ]:
# The way this API call works is we get job runs newest to oldest, so if you have 200 total runs and you pull 125 it will get the 125 most recent runs
last_125_job_runs = list_dbt_cloud_runs_for_a_job(dbt_cloud_api_token, dbt_cloud_account_id, dbt_cloud_job_id, max_runs=125)
print(last_125_job_runs)


___

# END OF SCRIPT

___